In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import cv2 as cv

import tensorflow as tf
import keras 
from keras_retinanet.preprocessing.csv_generator import CSVGenerator
from keras_retinanet.utils.transform import random_transform_generator
from keras_retinanet.models.resnet import resnet50_retinanet
from tensorflow.keras.models import load_model
from max_supression import retinanet_bbox
from sklearn.metrics import accuracy_score, precision_score
from keras_retinanet.utils.model import freeze
from tensorflow.keras.applications import ResNet101
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.image import preprocess_image, resize_image, read_image_bgr
from sklearn.metrics import precision_recall_curve
from keras_retinanet.utils.eval import evaluate
from keras_retinanet.models import convert_model
from keras_retinanet.losses import smooth_l1, focal


In [ ]:
## Data path root
ruta_datos = '.../path_root_images/'
## weights path
weights= '.../resnet101_oid_v1.0.0.h5'


In [ ]:
##defino le trasformazioni che si applicarano ai dati.
transform_generator= random_transform_generator(min_rotation=-0.2,
                                                max_rotation=0.2,
                                                min_shear=-0.1,
                                                max_shear=0.1,
                                                min_scaling=(0.7, 0.7),
                                                max_scaling=(1.3, 1.3),
                                                flip_x_chance=0.5)

In [ ]:
## costruimo i generatori di training e validazione

gen_train= CSVGenerator(ruta_datos+'path .....metadata_labels_retinanet_training.csv',
                       ruta_datos+'clase.csv',
                       transform_generator=transform_generator)

gen_val= CSVGenerator(ruta_datos+'path .....metadata_labels_retinanet_validation.csv',
                       ruta_datos+'clase.csv',
                       shuffle_groups=False)


In [ ]:
backbone = ResNet101(include_top=False, input_shape=(None, None, 3), weights='imagenet')

modifier = freeze
model = resnet50_retinanet(num_classes=1)

##load weigths
model.load_weights(weights, by_name=True, skip_mismatch=True)
model.summary()

In [ ]:
## clipnorm recomended (1e-5 y 0.001)

model.compile(loss={'regression': smooth_l1(),
                    'classification':focal()
                   }, 
              optimizer =tf.optimizers.Adam(learning_rate=0.00001, clipnorm=2),
              metrics=['accuracy']
             )

In [ ]:
history= model.fit(
    gen_train,
    epochs = 5,
    steps_per_epoch = 400,
    validation_data= gen_val
)

In [ ]:
##30 epch 110 stpvaldation
plt.figure(figsize=(15,8))

plt.subplot(241)
plt.plot(history.history['loss'], label='Training')
plt.plot(history.history['val_loss'], label='Validation')
plt.xlabel('Epochs')
plt.ylabel('Loss function')

plt.title("Loss function RetinaNet Stromboli")
plt.legend()
plt.grid()

plt.subplot(242)
plt.plot(history.history['regression_loss'], label='Training')
plt.plot(history.history['val_regression_loss'], label='Validation')
plt.xlabel('Epochs')
plt.ylabel('Regression loss')
plt.title("Regression loss RetinaNet Stromboli")
plt.legend()
plt.grid()

plt.subplot(243)
plt.plot(history.history['classification_loss'], label='Training')
plt.plot(history.history['val_classification_loss'], label='Validation')
plt.xlabel('Epochs')
plt.ylabel('Classification Loss ')
plt.title("Classification loss RetinaNet Stromboli")
plt.legend()
plt.grid()



In [ ]:

model_bbx= convert_model(model)
model_bbx_thr=retinanet_bbox(model, nms_threshold= 0.25)

In [ ]:


img= read_image_bgr('.../ path_validation_image')
draw= img.copy()
draw= cv.cvtColor(draw, cv.COLOR_BGR2RGB)
img= preprocess_image(img)
img, scale = resize_image(img)
img = np.expand_dims(img, axis=0)
boxes, scores, labels = model_bbx.predict(img)

boxes /= scale
## guardiamo le detections

for box, score, label in zip(boxes[0], scores[0], labels[0]):
    if score < 0.40:
        break
    color = label_color(label*13+3)
    b= box.astype(label)
    draw_box(draw,b,color=color,thickness=1 )
    nort_Explostion= label==0
    south_Explosion= label==1
    if nort_Explostion== True:
        caption= "{}{:.3f}".format('Explosion: ',score)
    elif south_Explosion== True:
        caption= "{}{:.3f}".format('Explosion: ',score)
    draw_caption(draw,b,caption)
    print(score,label)

plt.figure(figsize=(4,4))
plt.imshow(draw)


In [ ]:
from tensorflow.keras.models import save_model

#model_bbx_thr.save(".....path_save_model.h5")


In [ ]:
ap = evaluate(
    gen_val,
    model_bbx_thr,
    iou_threshold=0.6,
    score_threshold=0.05
)


print("The Average Precision (AP) of Iou 0.6 is: ", ap)
print("Averge Precision(AP) is:", format(100*( ap[0][0][0]),'.2f'),"%")

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_curve

def iou(a, b):
    x1,y1,x2,y2 = a; x1g,y1g,x2g,y2g = b
    xi1, yi1 = max(x1,x1g), max(y1,y1g)
    xi2, yi2 = min(x2,x2g), min(y2,y2g)
    inter = max(0, xi2-xi1) * max(0, yi2-yi1)
    ua = (x2-x1)*(y2-y1) + (x2g-x1g)*(y2g-y1g) - inter
    return 0.0 if ua <= 0 else inter / ua

all_tp, all_scores = [], []

for i in range(len(gen_val)):
    img = gen_val.load_image(i)
    ann = gen_val.load_annotations(i)['bboxes']

    img = preprocess_image(img)
    img, scale = resize_image(img)
    boxes, scores, labels = model_bbx_thr.predict_on_batch(np.expand_dims(img, axis=0))
    boxes /= scale

    used = np.zeros(len(ann), dtype=bool)

    for s, b in zip(scores[0], boxes[0]):
        best_iou, best_j = 0.0, -1
        for j, gt in enumerate(ann):
            if used[j]: 
                continue
            v = iou(b, gt)
            if v > best_iou:
                best_iou, best_j = v, j

        if best_iou >= 0.6 and best_j != -1:
            all_tp.append(1); used[best_j] = True
        else:
            all_tp.append(0)

        all_scores.append(float(s))

precision, recall, _ = precision_recall_curve(all_tp, all_scores)


In [ ]:
plt.figure(figsize=(15,8))
plt.title("Validation dataset")

plt.subplot(231)
plt.plot(history.history['loss'], label='Training')
plt.plot(history.history['val_loss'], label='Validation')
plt.xlabel('Epochs')
plt.ylabel('Loss function')

plt.title("Loss function RetinaNet Stromboli")
plt.legend()
plt.grid()

plt.subplot(232)
plt.plot(history.history['regression_loss'], label='Training')
plt.plot(history.history['val_regression_loss'], label='Validation')
plt.xlabel('Epochs')
plt.ylabel('Regression loss')
plt.title("Regression loss RetinaNet Stromboli")
plt.legend()
plt.grid()

plt.subplot(233)
plt.plot(recall, precision, label='Training')
plt.xlabel('Epochs')
plt.ylabel('')
plt.title("precision_recall_curve")
plt.legend()
plt.grid()


